In [1]:
import pandas as pd
import keywords as kwrd

In [2]:
indicators_map = kwrd.indicators
indicators_map.head()

,Goal,Target,UN Indicators
0,SDG 1,1.0,NaN
1,SDG 1,1.1,1.1.1
2,SDG 1,1.2,"1.2.1, 1.2.2"
3,SDG 1,1.3,1.3.1
4,SDG 1,1.4,"1.4.1, 1.4.2"


In [3]:
test_df = pd.read_excel('output/GD61/output/6-results/mapping_.xlsx', index_col=0)[['Goal', 'Target']]
test_df.head()

,Goal,Target
9,SDG 2,2.1
10,SDG 2,2.2
11,SDG 2,2.3
12,SDG 2,2.4
30,SDG 3,3.d


In [6]:
indicators_map = pd.merge(test_df, kwrd.indicators, how='left', on='Target').drop('Goal_y', axis=1).rename(columns={'Goal_x':'Goal'})
indicators_map.head()

,Goal,Target,UN Indicators
0,SDG 2,2.1,"2.1.1, 2.1.2"
1,SDG 2,2.2,"2.2.1, 2.2.2, 2.2.3"
2,SDG 2,2.3,"2.3.1, 2.3.2"
3,SDG 2,2.4,2.4.1
4,SDG 3,3.d,"3.d.1, 3.d.2"
